- This notebooks aims to find out the way to extract files from the database and decode them from latin-1 to text.
- Then create a dataframe out of the text


# Parsing english_subtitles_database.db file using sqlite3 library

## Step- 1 Understanding the .Db file

In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('../Database/eng_subtitles_database.db')
cursor = conn.cursor()

# Get the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the list of tables
for table in tables:
    print('Table Name:',table[0])

# Close the connection
conn.close()

Table Name: zipfiles


### Printing Top 10 rows in the table 

In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('../Database/eng_subtitles_database.db')
cursor = conn.cursor()

# Execute a query
cursor.execute('SELECT name,content FROM zipfiles Limit 10')
rows = cursor.fetchall()

# Process the results
for row in rows:
    print(row)

# Close the connection
conn.close()

('the.message.(1976).eng.1cd', b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07

## Step-2 Converting  database into a dataframe

In [3]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('../Database/eng_subtitles_database.db')

# Specify the table you want to convert to a pandas DataFrame
table_name = 'zipfiles'

# Read the table into a pandas DataFrame
df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

# Close the connection
conn.close()

# Now you can work with the data in the DataFrame
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB
None


,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


### Observation
- The table contains 82498 rows which is humguous.
- It mainly has 3 three columns
    - 'num'- Primary key
    - 'name'- Name of the tv show, movie or anime along with more info about that particular content
    - 'content'- Compressed latin-1 encoded transcripts of the shows(Looks like each row stores a zip file. So encoded transcripts are compressed)

# Step 3 Handling Duplicates

In [4]:
df.duplicated().sum()

0

In [5]:
# Let's check if there are duplicate names
df.duplicated(subset=['name']).sum()

30569

### Observation
- It was found that the rows containing duplicate names have content which slightly differ
- It is in best interest to remove these duplicate rows as it will reduce the dataset size from 80,000 to 50,000

In [6]:
#df=df.drop_duplicates(subset=['name'])


In [7]:
df.duplicated(subset=['name']).sum()

0

# Step-3 Extract content 

## Step-3.1 Extracting a single row to test the code

In [8]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[90, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text

ï»¿1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension: osdb.link/ext

2
00:00:24,107 --> 00:00:28,278
Legend has it,
in the mystic land of Prydain,

3
00:00:28,403 --> 00:00:31,865
there was once a king
so cruel and so evil

4
00:00:31,990 --> 00:00:35,035
that even the gods feared him.

5
00:00:35,160 --> 00:00:37,371
Since no prison could hold him,

6
00:00:37,496 --> 00:00:42,292
he was thrown alive
into a crucible of molten iron.

7
00:00:42,417 --> 00:00:45,462
There, his demonic spirit was captured

8
00:00:45,587 --> 00:00:50,050
in the form of a great Black Cauldron.

9
00:00:50,175 --> 00:00:54,346
For uncounted centuries,
the Black Cauldron lay hidden,

10
00:00:54,471 --> 00:00:58,392
waiting while evil men searched for it.

11
00:00:58,517 --> 00:01:00,727
Knowing whoever possessed it

12
00:01:00,853 --> 00:01:05,357
would have the power to resurrect
an army of deathless warriors.

13
00:01:06,275 --> 00:01:09,736
And, with th

### Observation
- Successfully able to decode a single zip file

## Step-3.2 Write a function to decode entire dataframe and decode it

In [9]:
def decode_text(bin_text):
    # Decompress the binary data using the zipfile module
    with io.BytesIO(bin_text) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Reading only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    text=subtitle_content.decode('latin-1')
    return text

In [10]:
# Applying to entire DataFrame
df['decoded_content']=df['content'].apply(decode_text)
df.head()

,num,name,content,decoded_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


### Observation
- Successfully decoded 51929 zip files and decoded to text

# Step-4 Save the file

In [12]:
new_df=df.drop(['num','content'],axis=1)
new_df

,name,decoded_content
0,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...,...
82490,immanence.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nSupport ..."
82493,the.prophets.game.(2000).eng.1cd,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,west.beirut.(1998).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,frankenstein.the.true.story.(1973).eng.1cd,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."


In [13]:
index=[i for i in range(len(new_df))]
new_df['index']=index
new_df

,name,decoded_content,index
0,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",0
1,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",1
2,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",2
3,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",3
4,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",4
...,...,...,...
82490,immanence.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nSupport ...",51924
82493,the.prophets.game.(2000).eng.1cd,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\...",51925
82494,west.beirut.(1998).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open...",51926
82495,frankenstein.the.true.story.(1973).eng.1cd,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati...",51927


In [15]:
#new_df=new_df.set_index('index')
new_df

,name,decoded_content
index,,
0,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...,...
51924,immanence.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nSupport ..."
51925,the.prophets.game.(2000).eng.1cd,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
51926,west.beirut.(1998).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."


In [ ]:
#save the file
#df.drop('content',axis=1).to_csv('../data/eng_subtitles.csv')

### Observation
- Schema of the resultant dataframe is modified based on future usage. 
- In future, we use index to find the vectors stored in vector DB. So explicitly created such index is neccessary
- As, data of dataframe exceeds 1GB. It is a best practice to remove unwanted data. Hence 'num' and 'content' columns were removed

# Step 6 Divide the file into pickle files containing 10,000 rows

In [ ]:
import pandas as pd
from tqdm import tqdm

# Assuming your DataFrame is called df
# Define the starting index and chunk size
start_index =0
chunk_size = 10000

# Calculate the number of chunks needed
num_chunks = ceil(len(new_df)/10000)

# Iterate over the chunks and save each chunk as a separate pickle file
for i in tqdm(range(num_chunks), desc="Chunking"):
    start_idx = start_index + i * chunk_size
    end_idx = min(start_index + (i + 1) * chunk_size, len(df))
    chunk = df[start_idx:end_idx]
    chunk.to_pickle(f'chunk_{i}.pkl')
